In [ ]:
import pandas as pd
import numpy as np
%ls

 adc.json                     indy_mailer_periods.csv
 indy_as400.csv               sample_data/
'Indy Mailed 6.10.2020.xls'  'Simpli.fi Indy Campaign June 1 to July 31.xlsx'
 indy_mailer2020.csv


In [ ]:

indym= pd.read_excel('Indy Mailed 6.10.2020.xls')
indym['AS400']= indym.AS400.astype('str').unique()
print(indym.AS400.astype('str').tolist())

['5108462', '5108247', '5100103', '5100062', '5109028', '5109035', '5108597', '19107', '5108587', '5108470', '5108010', '18010', '5108725', '19983', '5119000', '19166', '5108991', '18013', '18015', '5109865', '18324', '19638', '5109873', '19359', '5108828', '5108018', '19833', '18020', '19623', '5108234', '5109328', '5120051', '5109381', '5108662', '5108528', '19015', '19264', '5108185', '18742', '19161', '5108908', '19076', '5109830', '19256', '19121', '18029', '5107618', '19148', '5109888', '5109851', '5109838', '5109329', '19143', '5108480', '5107612', '5108479', '19197', '5109807', '5109004', '5108473', '19826', '18035', '18445', '5107652', '19268', '18037', '19827', '19964', '5109023', '5108936', '18039', '5109025', '5109849', '5108918', '19708', '5107627', '5108957', '5108487', '5109869', '18937', '5109845', '5107611', '18041', '18048', '5109857', '18045', '5100041', '5107629', '5108493', '5109475', '5109825', '18049', '19975', '5109829', '18051', '5108827', '18070', '5110725', '

In [ ]:
indyr= pd.read_excel('Simpli.fi Indy Campaign June 1 to July 31.xlsx')

In [ ]:
indyr['AS400']= indyr.loc[:, 'Geography Geo Fence Name'].apply(lambda x: x[x.rfind('-'):]).str.replace('-','').str.strip().astype(str)

In [ ]:
len(indym.AS400.unique())

566

In [ ]:
campaign= indym.merge(indyr, on='AS400', how='left').isna().sum()

In [ ]:
campaign.to_csv('indy_mailer2020.csv')

In [ ]:
df_col= ['datetime', 'machine_id', 'machine_type', 'room_id', 'as400id', 'payment_type', 'usr_id', 'amount', 'txn_num_daily']

In [ ]:
df = pd.read_csv('indy_as400.csv', header=None, names= df_col)

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df['datetime']= pd.to_datetime(df.datetime)
df['as400id']=df.as400id.astype('str')

In [ ]:
df.shape

(3572169, 9)

In [ ]:
df= df.loc[df.datetime <= pd.datetime.now()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [ ]:
df1= df.loc[df.payment_type=='App']

In [ ]:
df1.shape

(1667444, 10)

In [ ]:
df1.head()

,datetime,machine_id,machine_type,room_id,as400id,payment_type,usr_id,amount,txn_num_daily,period
56,2016-02-29,011-BBM,Dryer,DI-5108779-78,5108779,App,1014402,1.85,1,None
57,2016-02-29,121-HWX,Dryer,DI-5109901-1,5109901,App,469489,1.50,1,None
58,2016-02-29,121-HWX,Dryer,DI-5109901-1,5109901,App,57691,1.50,1,None
63,2016-03-08,121-HWX,Dryer,DI-5109901-1,5109901,App,38891,1.50,1,None
82,2019-06-14,346-HNY,Washer,DI-19268-1,19268,App,33373,2.00,1,None


In [ ]:
df1.dtypes

datetime         datetime64[ns]
machine_id               object
machine_type             object
room_id                  object
as400id                  object
payment_type             object
usr_id                   object
amount                  float64
txn_num_daily             int64
period                   object
dtype: object

In [ ]:
df1['period']= None

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1.loc[df1.datetime.between('2020-04-10', '2020-06-09'), 'period']= 'pre campaign'
df1.loc[df1.datetime.between('2020-06-10', '2020-08-09'), 'period']= 'campaign'
df1.loc[df1.datetime.between('2020-08-10', '2020-10-09'), 'period']= '2nd mailer'
df1.loc[df1.datetime.between('2020-10-10', '2020-12-09'), 'period']= 'post campaign'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
len(df1.usr_id.unique())

64515

In [ ]:
vend_range= df1.groupby('usr_id')['datetime'].agg(['min', 'max']).reset_index().rename(columns={'usr_id': 'usr_id','min':'first_vend', 'max':'last_vend'})

In [ ]:
df1= df1.merge(vend_range, on='usr_id', how='left')

In [ ]:
df1['first_vend_period']= None
df1.loc[df1.first_vend.between('1900-04-10', '2020-04-09'), 'first_vend_period']= 'before'
df1.loc[df1.first_vend.between('2020-04-10', '2020-06-09'), 'first_vend_period']= 'pre campaign'
df1.loc[df1.first_vend.between('2020-06-10', '2020-08-09'), 'first_vend_period']= 'campaign'
df1.loc[df1.first_vend.between('2020-08-10', '2020-10-09'), 'first_vend_period']= '2nd mailer'
df1.loc[df1.first_vend.between('2020-10-10', '2020-12-09'), 'first_vend_period']= 'post campaign'
df1.loc[df1.first_vend.between('2020-12-10', '2021-08-09'), 'first_vend_period']= 'after'

In [ ]:
df1['last_vend_period']= None
df1.loc[df1.last_vend.between('1900-04-10', '2020-04-09'), 'last_vend_period']= 'before'
df1.loc[df1.last_vend.between('2020-04-10', '2020-06-09'), 'last_vend_period']= 'pre campaign'
df1.loc[df1.last_vend.between('2020-06-10', '2020-08-09'), 'last_vend_period']= 'campaign'
df1.loc[df1.last_vend.between('2020-08-10', '2020-10-09'), 'last_vend_period']= '2nd mailer'
df1.loc[df1.last_vend.between('2020-10-10', '2020-12-09'), 'last_vend_period']= 'post campaign'
df1.loc[df1.last_vend.between('2020-12-10', '2021-08-09'), 'last_vend_period']= 'after'

In [ ]:
pd.crosstab(df1.first_vend_period, df1.last_vend_period, df1.usr_id, aggfunc='nunique', margins=True).\
reindex(['before', 'pre campaign', 'campaign', '2nd mailer', 'post campaign','after' ], axis='rows').\
reindex(['before', 'pre campaign', 'campaign', '2nd mailer', 'post campaign','after' ], axis='columns')

In [ ]:
df1.isna().sum()

datetime                  0
machine_id                0
machine_type              0
room_id                   0
as400id                   0
payment_type              0
usr_id                    0
amount                    0
txn_num_daily             0
period               910160
first_vend                0
last_vend                 0
first_vend_period         0
last_vend_period          0
dtype: int64

In [ ]:
# df1= df.dropna(axis=0)

In [ ]:
df1.to_csv('indy_mailer_periods.csv')

In [ ]:
df1.loc[df1.last_vend_period.isnull()]

,datetime,machine_id,machine_type,room_id,as400id,payment_type,usr_id,amount,txn_num_daily,period,first_vend,last_vend,first_vend_period,last_vend_period
3,2016-02-29,121-HWX,Dryer,DI-5109901-1,5109901,Coin,99999999,3.00,7,None,2016-02-29,2079-02-15,before,None
4,2016-03-01,121-HWX,Dryer,DI-5109901-1,5109901,Coin,99999999,4.50,3,None,2016-02-29,2079-02-15,before,None
5,2016-03-03,121-HWX,Dryer,DI-5109901-1,5109901,Coin,99999999,1.50,1,None,2016-02-29,2079-02-15,before,None
6,2016-03-06,121-HWX,Dryer,DI-5109901-1,5109901,Coin,99999999,1.50,1,None,2016-02-29,2079-02-15,before,None
8,2016-03-15,121-HWX,Dryer,DI-5109901-1,5109901,Coin,99999999,1.50,1,None,2016-02-29,2079-02-15,before,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3572007,2021-04-19,004-HWW,Washer,DI-19803-1,19803,Coin,99999999,1.75,1,None,2016-02-29,2079-02-15,before,None
3572044,2021-04-19,247-HTZ,Dryer,DI-18996-2,18996,Coin,99999999,2.50,4,None,2016-02-29,2079-02-15,before,None
3572114,2021-04-19,578-HWX,Washer,DI-18097-1,18097,Coin,99999999,1.00,1,None,2016-02-29,2079-02-15,before,None
3572129,2021-04-19,740-HWY,Dryer,DI-5108932-20,5108932,Coin,99999999,3.50,2,None,2016-02-29,2079-02-15,before,None


In [ ]:
df.datetime.max()

Timestamp('2079-02-15 00:00:00')